<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/preprocess_to_aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with PyTorch
## 3D Convolutional Neural Network on Group Brain fMRI
Contributors: Stacey Rivet Beck, Ben Merrill
### To Do:
- Either:
  - Get Raw data in 4D   
          - OR -
  - Reshape Raw data into 4D from 2D
    - Apply Whole Brain Mask to data and save to AWS

- Build Dataloader: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

- Implement 3DCNN from paper: REALLY GREAT PAPERS
  - Nguyen et al. http://proceedings.mlr.press/v136/nguyen20a/nguyen20a.pdf
  - Wang et al. https://arxiv.org/pdf/1801.09858.pdf (discusses more in detail the input data shapes and processing)
  
  - Inputs: 84@ x * y * z ; one fmri time series at a time, not concatenated
  - Basic Architecture:
        #First layer is generating temporal descriptors of the voxels
        Conv1 1 x 1 x 1 filter, output = 32, stride = 1, ReLU, BatchNorm
        Conv2 7 x 7 x 7 filter, output = 64, stride = 2, ReLU, BatchNorm
        Conv3 3 x 3 x 3 fitler, output = 64, stride = 2, ReLU, BatchNorm
        Conv4 3 x 3 x 3 fitler, output = 128, stride = 2, ReLU, BatchNorm
        Global Average Pooling on final feature maps ->
        Flattened maps size 128?
        Fully connected layer size 64
        Fully connected layer size 2 (2 way classification, one for each class) -> softmax

        Optimized with Adam, standard parameters (β1=0.9 and β2=0.999)
        Batched at 32, but we may need to batch smaller due to GPU compute
        Learning Rate = 0.001, gradual decay after Val loss plateaued after 15 epochs
        Cross entropy Loss
        Employ early stopping
        In Wang et al. they used data for visualization, same size as input data, though are reduced in time dimension to be mapped on fsaverage surface. 
        
  

## Importing Dataset and Labels

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git

# Change directory into cloned repo DL folder
%cd teambrainiac/source/DL

# !ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 1152, done.
remote: Counting objects: 100% (1152/1152), done.
remote: Compressing objects: 100% (900/900), done.
remote: Total 1152 (delta 731), reused 474 (delta 236), pack-reused 0
Receiving objects: 100% (1152/1152), 83.61 MiB | 29.18 MiB/s, done.
Resolving deltas: 100% (731/731), done.
/content/teambrainiac/source/DL


### Load path_config.py

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 196 bytes


## Import Packages

In [4]:
# Possible Missing Packages
!pip install boto3

     |████████████████████████████████| 132 kB 30.1 MB/s 
     |████████████████████████████████| 8.7 MB 64.4 MB/s 
     |████████████████████████████████| 79 kB 9.6 MB/s 
     |████████████████████████████████| 138 kB 74.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
# General Library Imports
import re
import scipy.io
import os
import pickle
import numpy as np
import nibabel as nib
import pandas as pd
import boto3
import tempfile
import tqdm
import random
from path_config import mat_path
from botocore.exceptions import ClientError
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

# From Local Directory
from access_data_dl import *
from process_dl import *

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

#import torchvision.transforms as transforms
from torch.nn import ReLU, CrossEntropyLoss, Conv3d, Module, Softmax, AdaptiveAvgPool3d
from torch.optim import Adam, SGD

#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
# from dataloader_class import DatasetFmri


## Import Group fMRI Data, Normalize, and Create Masks

In [6]:
# Open path dictionary file to get subject ids
path = "../data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)

In [8]:

'''
This is to keep hardcoded dictionary of shuffled ids to make sure train, validation, and test sets stay seperate

If the size you are looking for is not here, use the function at the bottom of the cell.

five_ids = {'test': ['10017_08894'],
 'train': ['10008_09924', '10016_09694', '10004_08693'],
 'val': ['10009_08848']}


all_ids = {'test': ['10056_09615','10035_08847','10038_09063','30044_10095','10080_09931',
  '30017_09567','10084_10188','10069_09785','10061_09308','10039_08941','10004_08693'],
 'train': ['30012_09102','30027_09638','30033_09776','10033_08871',
  '10057_10124','30036_09758','10036_09800','10047_09030','30025_09402','10043_09222','30024_09398','10066_09687',
  '10023_09126','10050_09079','10022_08854','30053_10112','10016_09694','30011_09170','30035_09836','10065_09587',
  '10045_08968','10008_09924','10060_09359','30014_09352','10018_08907','30020_09236','10027_09455','10046_09216',
  '30045_10182','30038_09967','10037_09903','30009_09227','10021_08839','30004_08965','30008_08981','10053_09018'],
 'val': ['10034_08879','10042_08990','10009_08848','10017_08894','30026_09430']}

'''

# generate_train_val_test_dict(subject_id_partition, train_val_test_proportion=[0.7,0.8,1])

In [ ]:
all_ids = {'test': ['10056_09615','10035_08847','10038_09063','30044_10095','10080_09931',
  '30017_09567','10084_10188','10069_09785','10061_09308','10039_08941','10004_08693'],
 'train': ['30012_09102','30027_09638','30033_09776','10033_08871',
  '10057_10124','30036_09758','10036_09800','10047_09030','30025_09402','10043_09222','30024_09398','10066_09687',
  '10023_09126','10050_09079','10022_08854','30053_10112','10016_09694','30011_09170','30035_09836','10065_09587',
  '10045_08968','10008_09924','10060_09359','30014_09352','10018_08907','30020_09236','10027_09455','10046_09216',
  '30045_10182','30038_09967','10037_09903','30009_09227','10021_08839','30004_08965','30008_08981','10053_09018'],
 'val': ['10034_08879','10042_08990','10009_08848','10017_08894','30026_09430']}

In [9]:
# To avoid an insane amount of RAM, we will take the all_ids dictionary and split it up into chunks
# Test and val are reasonably sized, but train is not. We will split train into 4 pieces
train_len = len(all_ids['train'])
all_ids['train_1'] = all_ids['train'][:int(train_len/4)]
all_ids['train_2'] = all_ids['train'][int(train_len/4):int(train_len/2)]
all_ids['train_3'] = all_ids['train'][int(train_len/2):int(3*train_len/4)]
all_ids['train_4'] = all_ids['train'][int(3*train_len/4):]

del all_ids['train']

In [ ]:
# labels_mask_binary hyperparameters
label_type='rt_labels'

# load_subjects_by_id parameters
n_subjects = len(data_path_dict['subject_ID'])
runs = [2,3]

# get_mask parameters and pull mask
image_mask_type = 'mask'
mask_ind = 0
brain_mask = get_mask(image_mask_type, data_path_dict, mask_ind)

# mask_normalize_runs_reshape_4d parameters
scaler = 'standard'



for subject_partition in all_ids.keys():
  subject_ids = all_ids[subject_partition]

  image_label_mask, image_labels = labels_mask_binary(data_path_dict, label_type='rt_labels')

  initial_subject_data = load_subjects_by_id(data_path_dict, subject_ids, image_label_mask, image_labels, label_type, runs)

  subjects_reshaped = mask_normalize_runs_reshape_4d(initial_subject_data, brain_mask, scaler)

  partition_dictionary = train_test_aggregation_group(subjects_reshaped, runs, subject_ids)

  s3_upload(partition_dictionary, 'dl/partition_%s.pkl'%subject_partition, 'pickle')
  print('Partition', subject_partition, 'complete.')
  print()

Subject ids loaded.
Adding subjects to dictionary.


11it [02:13, 12.12s/it]


Completed Subject 1
Completed Subject 2
Completed Subject 3
Completed Subject 4
Completed Subject 5
Completed Subject 6
Completed Subject 7
Completed Subject 8
Completed Subject 9
Completed Subject 10
Completed Subject 11
upload complete for dl/partition_test.pkl
Partition test complete.

Subject ids loaded.
Adding subjects to dictionary.


5it [00:59, 11.97s/it]


Completed Subject 1
Completed Subject 2
Completed Subject 3
Completed Subject 4
Completed Subject 5
upload complete for dl/partition_val.pkl
Partition val complete.

Subject ids loaded.
Adding subjects to dictionary.


9it [01:48, 12.08s/it]


Completed Subject 1
Completed Subject 2
Completed Subject 3
Completed Subject 4
Completed Subject 5
Completed Subject 6
Completed Subject 7


## Build Dataloader

In [ ]:
def get_dataloader(train_dict, test_dict, bs):
  '''
  train_dict : Dictionary of training images and their labels
   
  test_dict  : Dictionary of testing images and their labels
  '''

    return (
        DataLoader(train_ds, batch_size=bs),#, shuffle=True),
        DataLoader(test_ds, batch_size=bs * 2)
    )

In [ ]:
from torch.utils.data import TensorDataset

x_train = train_dict['images']
y_train = train_dict['labels']

x_test = test_dict['images']
y_test = test_dict['labels']

train_ds = TensorDataset(x_train, y_train)
test_ds = TensorDataset(x_test, y_test)

In [ ]:
bs = 6

train_dl = DataLoader(train_ds, batch_size=bs)
test_dl = DataLoader(test_ds, batch_size=bs*2)

In [ ]:
# bs = 6

# train_dl, test_dl = get_dataloader(train_ds, test_ds, bs)

In [ ]:
# Sanity Check
train_features, train_labels = next(iter(train_dl))
test_features, test_labels = next(iter(test_dl))

## Practice Model

torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 64, 37, 45, 37])
torch.Size([6, 64, 18, 22, 18])
before drop torch.Size([6, 128, 1, 1, 1])
after drop torch.Size([6, 128])
after lin layer torch.Size([6, 64])
torch.Size([6, 2])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 64, 37, 45, 37])
torch.Size([6, 64, 18, 22, 18])
before drop torch.Size([6, 128, 1, 1, 1])
after drop torch.Size([6, 128])
after lin layer torch.Size([6, 64])
torch.Size([6, 2])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 32, 79, 95, 79])
torch.Size([6, 64, 37, 45, 37])
torch.Size([6, 64, 18, 22, 18])
before drop torch.Size([6, 128, 1, 1, 1])
after drop torch.Size([6, 128])
after lin layer torch.Size([6, 64])
torch.Size([6, 2])


KeyboardInterrupt: ignored

## Build Model

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    #Conv1
    self.conv1 = nn.Conv3d(in_channels = 1, 
                           out_channels = 32, 
                           kernel_size = (1,1,1), 
                           stride = (1,1,1)
                           )
    self.bn1 = nn.BatchNorm3d(32)
    self.conv2 = nn.Conv3d(in_channels = 32, 
                           out_channels = 64, 
                           kernel_size = (7,7,7),
                           stride = (2,2,2)
                           )
    self.bn2 = nn.BatchNorm3d(64)
    self.conv3 = nn.Conv3d(in_channels = 64, 
                           out_channels = 64, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn3 = nn.BatchNorm3d(64)
    self.conv4 = nn.Conv3d(in_channels = 64, 
                           out_channels = 128, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn4 = nn.BatchNorm3d(128) 
    self.pool1 = nn.AdaptiveAvgPool3d((1,1,1)) #Global Average Pool, takes the average over last two dimensions to flatten 
  
                                                             
    # Fully connected layer
    self.fc1 = nn.Linear(128,64) # need to find out the size where AdaptiveAvgPool 
    self.fc2 = nn.Linear(64, 2) # left with 2 for the two classes                     



  def forward(self, xb):
    xb = self.bn1(F.relu(self.conv1((xb))))
    xb = self.bn2(F.relu(self.conv2((xb)))) # Takes a long time
    xb = self.bn3(F.relu(self.conv3((xb))))
    xb = self.bn4(F.relu(self.conv4((xb))))
    xb = self.pool1(xb)
    xb = xb.view(xb.shape[:2])
    xb = self.fc1(xb)
    xb = self.fc2(xb)
    return xb      

    

In [ ]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
print("First model training on GPU")
print(model)

# Initialize other parameters
epochs = 3 #120
learning_rate = 0.0002
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

First model training on GPU
ConvNet(
  (conv1): Conv3d(1, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv3d(32, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2))
  (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (bn3): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (bn4): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
# Run Model
for epoch in range(1, 1+epochs):
  model.train()
  i = 1
  print('epoch', epoch)
  for xb, yb in train_dl:
    print('batch', i)
    i = i+1

    xb = xb.float()
    pred = model(xb)
    loss = loss_func(pred, yb)
    print('Loss', loss)

    loss.backward()
    opt.step()
    opt.zero_grad()

  model.eval()

  print(epoch, valid_loss / len(valid_dl))

epoch 1
batch 1
Loss tensor(0.6779, grad_fn=<NllLossBackward0>)
batch 2
Loss tensor(0.6765, grad_fn=<NllLossBackward0>)
batch 3
Loss tensor(0.6590, grad_fn=<NllLossBackward0>)
batch 4
Loss tensor(0.7258, grad_fn=<NllLossBackward0>)
batch 5
Loss tensor(0.6836, grad_fn=<NllLossBackward0>)
batch 6
Loss tensor(0.7156, grad_fn=<NllLossBackward0>)
batch 7
Loss tensor(0.7124, grad_fn=<NllLossBackward0>)
batch 8
Loss tensor(0.7137, grad_fn=<NllLossBackward0>)
batch 9
Loss tensor(0.7218, grad_fn=<NllLossBackward0>)
batch 10
Loss tensor(0.6575, grad_fn=<NllLossBackward0>)
batch 11
Loss tensor(0.7254, grad_fn=<NllLossBackward0>)
batch 12
Loss tensor(0.6792, grad_fn=<NllLossBackward0>)
batch 13
Loss tensor(0.6600, grad_fn=<NllLossBackward0>)
batch 14
Loss tensor(0.7117, grad_fn=<NllLossBackward0>)


NameError: ignored

## Training

In [ ]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
print("First model training on GPU")
print(model)

# Initialize other parameters
epochs = 5 #120
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

fit(epochs, model, )

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [ ]:
accuracy_stats = {
    'train': [],
    'val': []
  }

print(accuracy_stats)

loss_stats = {
    'train': [],
    'val': []
    }
print(loss_stats)



def train_test_model(epochs, model, opt, train_dl, test_dl, device):
  for epoch in range(1, epochs+1):

    # TRAINING *****************************************************************

    train_epoch_loss = 0
    train_epoch_acc = 0

    # set model in training mode 
    model.train()
    print('\nEpoch$ : %d'%epoch)
    for xb, yb in tqdm(train_dl):
      xb = xb.to(device)
      y_train_batch = y_train_batch.to(device) 

      #print(x_train_batch.shape)

      # sets gradients to 0 to prevent interference with previous epoch
      optimizer.zero_grad()
    
      # Forward pass through NN
      y_pred = model(xb)#.to(float)
      train_loss = criterion(y_pred, yb)
      train_acc = accuracy(y_pred, yb)

      # Backward pass, updating weights
      train_loss.backward()
      opt.step()

      # Statistics
      train_epoch_loss += train_loss.item()
      train_epoch_acc += train_acc.item()


    # VALIDATION****************************************************************   
    
    with torch.set_grad_enabled(False):
      val_epoch_loss = 0
      val_epoch_acc = 0

      model.eval()
      for x_val_batch, y_val_batch in tqdm(validate_loader):
      
        x_val_batch =  x_val_batch.to(device)#.to(float)
        y_val_batch = y_val_batch.to(device)
            
        # Forward pass
        y_val_pred = model(x_val_batch)#.to(float)   
        val_loss = criterion(y_val_pred, y_val_batch)
        val_acc = accuracy(y_val_pred, y_val_batch)
            
        val_epoch_loss += val_loss.item()
        val_epoch_acc += val_acc.item()

    # Prevent plateauing validation loss 
    #scheduler.step(val_epoch_loss/len(validate_loader))

        
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(validate_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(validate_loader))
                              
    
    print(f'Epoch {epoch+0:03}: Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(validate_loader):.5f}') 
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f} | Val Acc: {val_epoch_acc/len(validate_loader):.3f}')

      




# def accuracy(y_pred, y_test):
#   # Calculating model accuracy at each epoch 
#   y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
#   _, y_pred_prob = torch.max(y_pred_softmax, dim = 1)
#   correct_pred = (y_pred_prob == y_test).float()
#   acc = correct_pred.sum() / len(correct_pred)
#   acc = torch.round(acc * 100)

#   return acc



     





# if __name__ == '__main__':
#   train_val_model(epochs)

{'train': [], 'val': []}
{'train': [], 'val': []}

Epoch$ : 1


NameError: ignored

In [ ]:
model = ConvNet()

epochs = 5 #120
learning_rate = 0.001
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)



for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
      xb = xb.float()
      loss_batch(model, loss_func, xb, yb, opt)

    model.eval()
    # with torch.no_grad():
    #     losses, nums = zip(
    #         *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
    #     )
    # val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

    # print(epoch, val_loss)